In [1]:
import pandas as pd

movie_df= pd.read_json("/content/movie.json")
rating_df= pd.read_json("/content/rating.json")

FileNotFoundError: ignored

In [ ]:
movie_df.head(2)

In [ ]:
rating_df.head(2)

In [ ]:
movie_df.rename( columns={'id': 'movieId'}, inplace=True)

In [ ]:
movie_df

In [ ]:
# merge : 데이터 2개를 합친다, 3개를 합치려면 2개 합치고 나중에 1개를 추가
# 컬럼명이 동일해야 한다. 양 쪽에 동일하지 않다면 수정한다.
# 양 쪽에 똑같이 밸류 값이 있지 않으면 유실 됨, 방지하려면 how 함수

movie_rating_df= pd.merge(movie_df, rating_df, on= 'movieId', how= 'left'); movie_rating_df # how= left(null까지 다 가져오기)

# on = 연결고리(공통컬럼), how= left 하면 왼쪽 데이터 다 살리고, 한 쪽에 없는 값은 nan으로 처리해줌

In [ ]:
# correlation(상관계수)을 구하기 위해 pivot_table 사용한다.
df= movie_rating_df.pivot_table(index= 'userId', columns= 'title', values= 'rating'); df

In [ ]:
# correlation(상관계수)을 확인한다. 별점 준 사람이 20명이 넘는 내린 것으로 corr 해봐라.
corr_movie = df.corr(min_periods=20); corr_movie

In [ ]:
# # 특정 유저의 맞춤형 영화 추천
# 'userId' 3번 사람의 예
rating_df['userId']==3
user_3= rating_df.loc[rating_df['userId']==3, ] # [컬럼, 비우는 이유]

In [ ]:
# user_3에다가 영화제목 붙여라. title과 rating만 있는 데이터 변수를 생성
user_3_df= pd.merge(user_3, movie_df, on= 'movieId')[ ['title', 'rating'] ]

In [ ]:
recom_movies = corr_movie['Killing Room, The'].dropna().sort_values(ascending=False).to_frame()

In [ ]:
# crrelation 컬럼 생성
recom_movies.columns = ['correlation']

In [ ]:
recom_movies['weight'] = recom_movies['correlation'] * user_3_df['rating'][0]

In [ ]:
similar_movies_list = pd.DataFrame()
for i in range(user_3_df.shape[0]) : # 유저가 본 영화의 행의 갯수
    movie_title = user_3_df['title'][i]
    recom_movies = corr_movie[movie_title].dropna().sort_values(ascending=False).to_frame()
    recom_movies.columns = ['correlation']
    recom_movies['weight'] = recom_movies['correlation'] * user_3_df['rating'][i]
    similar_movies_list= pd.concat([similar_movies_list,recom_movies])

# Weight(가중치)는 상관계수 * 점수

In [ ]:
similar_movies_list
similar_movies_list.sort_values('weight', ascending = False, inplace = True)

In [ ]:
user_3_df

In [ ]:
# 내가 본 영화의 제목을 가져온다
for name in user_3_df['title'] :
    if name in similar_movies_list.index :
        similar_movies_list.drop(name, axis = 0, inplace=True)

In [ ]:
similar_movies_list

In [ ]:
# 중복 된 영화 제거 (타이틀)
similar_movies_list.reset_index()['title']
similar_movies_list.reset_index()['title'].value_counts() # 중복갯수 구하기

In [ ]:
# 추천 영화 10개 보기
similar_movies_list
similar_movies_list.groupby('title')['weight'].max()
similar_movies_list.groupby('title')['weight'].max().sort_values(ascending=False).head(10)
similar_movies_list.groupby('title')['weight'].max().sort_values(ascending=False).head(10).to_frame() #df로 변환
recom_df = similar_movies_list.groupby('title')['weight'].max().sort_values(ascending=False).head(10).to_frame().reset_index() ; recom_df

In [ ]:
# 타이틀 칼럼이 같은 것 끼리 합친다.
pd.merge(recom_df, movie_df, on= 'title')
recom_df= pd.merge(recom_df, movie_df, on= 'title'); recom_df

In [ ]:
# 이거만 있으면 가능할걸
# - get http://127.0.0.1:5000/movie/recommend
# - header : JWT Token
# 클라이언트가 api 호출 -> header Token =user_id
corr_movie

In [ ]:
corr_movie.to_csv('corr_movie.csv')
# 좌측 파일쪽 새로고침해서 다운로드